In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Tower_data\RTdatMaster20221121.csv'
dtypes = {'Date_QA': 'str', 'Time': 'str', 'DateAndTime': 'str'}
parse_dates = ['Date_QA', 'DateAndTime']

RTdat = pd.read_csv(data_path, dtype=dtypes, parse_dates=parse_dates, dayfirst=True)
# RTdat = RTdat.loc[RTdat['TagID']==70] # for testing, to be commented out
# RTdat

In [3]:
data_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Simul_data\Simul_data_20230202.xlsx'
SIMdat = pd.read_excel(data_path)
# SIMdat

In [4]:
#Train test split of simulation data
seed = 38 # Seed for train/test split
test_size = 0.2

SIMdat_train, SIMdat_test = train_test_split(SIMdat, test_size=test_size, random_state=seed)

Add Latitude and longitude to RTdat to identify simulation data

In [5]:
def append_coordinates(RTdat, SIMdat):
    RTdat_sim = RTdat.copy()
    for i, (index, row) in enumerate(RTdat_sim.iterrows()):
        sim_query = SIMdat.query("Start_time <= @row.DateAndTime and End_time >= @row.DateAndTime and Tag_ID == @row.TagID")
        if not sim_query.empty:
            d = sim_query.Data_type.iloc[0]
            x = sim_query.POINT_X.iloc[0]
            y = sim_query.POINT_Y.iloc[0]
            RTdat_sim.loc[index, 'Data_type'] = d
            RTdat_sim.loc[index, 'POINT_X'] = x
            RTdat_sim.loc[index, 'POINT_Y'] = y

        if i % 10000 == 0:
            print(i)
    RTdat_sim = RTdat_sim.loc[~RTdat_sim['Data_type'].isna()]
    return RTdat_sim

In [ ]:
RTdat_sim_train = append_coordinates(RTdat, SIMdat_train)
RTdat_sim_test = append_coordinates(RTdat, SIMdat_test)

In [ ]:
output_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Simul_data\Processed_RTdat_w_LatLong_append'
train_output_name = '\RTdat_sim_train_20230212.xlsx'
test_output_name = '\RTdat_sim_test_20230212.xlsx'

RTdat_sim_train.to_excel(output_path+train_output_name, index=False)
RTdat_sim_test.to_excel(output_path+test_output_name, index=False)